In [47]:
%matplotlib inline

from matplotlib import pyplot as plt
from ipywidgets import interactive, HBox, VBox, Layout
import numpy as np

precio_compra = 76_000
mensualidad = 450
capital = 35_000
por_pagar = precio_compra
alexis = 0

tooltips = {
   'capital'     : 'prestamo de Vicen',
   'mes_pago'    : 'mes en el que se acaba de pagar el bajo',
   'obra'        : 'precio de la obra',
   'pagos_extras': 'pagos al ayuntamiento, arquiteco etc...',
   'aire_etc'    : 'aire acondicionado, camillas otro material',
   'gastos'      : 'luz, agua, internet...',
   'pacientes'   : 'numero de pacientes a la semana',
   'consulta'    : 'precio (medio) de la consulta',
   'sueldo'      : 'nomina de Alexis',
   'devolucion'  : 'devolucion del prestamo a Vicen',
}

def f(capital, pacientes=30, consulta=30, obra=32000, pagos_extras=2000, 
      aire_etc=4000, gastos=300, sueldo=1500, devolucion=200, mes_pago=36):
    fig = plt.figure(2, figsize=(14, 10))
    axe = plt.subplot(211)
    global por_pagar
    por_pagar = precio_compra
    base_cotisacion = 944.40
    ys = {
        'cuota autonomo'     : [],
        'sueldo'    : [],
        'gastos'    : [],
        'ingresos'  : [],
        'devolucion Vicen': [],
        'alquiler'  : []
    }
    y = []
    for month in range(5 * 12 + 1):
        if month < 2:
            cuota = 51.50 + 12.28 + 0.009 * base_cotisacion + 0.001 * base_cotisacion
        else:
            cuota = 0.283 * base_cotisacion + 0.013 * base_cotisacion + 0.009 * base_cotisacion
            if month < 14:
                cuota *= 0.5
            elif month < 20:
                cuota *= 0.7
        capital -= devolucion
        capital -= cuota
        ys['cuota autonomo'].append(cuota)
        ys['devolucion Vicen'].append(devolucion)
        if month <= 4:
            ingreso = pacientes * 4 * 20
            capital += ingreso
            capital -= sueldo
            ys['gastos'].append(0)
            if month == 4:
                plt.text(2, -7000, 'casa\nVicen', ha='center')
                plt.plot([0, 4], [-8000, -8000], lw=5, color='tab:orange')
        else:
            ingreso = pacientes * 4 * consulta
            capital += ingreso
            capital -= gastos
            ys['gastos'].append(gastos)
            capital -= sueldo
            if month == 5:
                plt.text(20, -7000, 'en el bajo: descuentas gastos pero ya no cobras 20 euros', ha='center')
                plt.plot([4, 5*12], [-8000, -8000], lw=5, color='tab:green')
        ys['ingresos'].append(ingreso)
        ys['sueldo'].append(sueldo)
        if month == 1:
            capital -= pagos_extras
            plt.text(month, 50_000, 'Extras', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month == 2:
            capital -= obra
            plt.text(month, 50_000, 'obra', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month == 4:
            capital -= aire_etc
            plt.text(month, 50_000, 'aire', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month == mes_pago:
            capital -= por_pagar
            plt.text(month, 50_000, 'bajo', rotation=90)
            plt.plot([month, month], [capital, 50_000], color='tab:grey')
        if month < mes_pago:
            por_pagar -= mensualidad
            ys['alquiler'].append(mensualidad)
        else:
            ys['alquiler'].append(0)

        y.append(capital)
    x = range(5 * 12 + 1)
    
    y = np.array(y)
    
    plt.fill_between(x, y, where=y<0, interpolate=True, color='tab:red')
    plt.fill_between(x, y, where=y>0, interpolate=True, color='tab:blue')

    plt.xticks(range(4, 5 * 12 + 4, 12), [2022 + i for i in range(0, 5)])
    plt.ylabel('diners (cumulativo)')
    axe.set_xticks(range(5 * 12), minor=True)
    plt.grid()
    plt.grid(which='minor', alpha=0.2)
    plt.ylim(-10_000, 60_000)

    axe = plt.subplot(212, sharex=axe)

    for k in ys:
        ys[k] = np.array(ys[k])

    plt.fill_between(x, ys['ingresos'], label='ingresos')
    prev = np.zeros(len(ys['sueldo']))
    for k in ['cuota autonomo', 'sueldo', 'gastos', 'devolucion Vicen', 'alquiler']:
        plt.fill_between(x, -ys[k] - prev, -prev,  label=k)
        prev += ys[k]

    plt.plot(x, ys['ingresos'] - prev, '--', lw=2, color='tab:red', label='diferencia')

    plt.legend(loc=1)
    plt.xticks(range(4, 5 * 12 + 4, 12), [2022 + i for i in range(0, 5)])
    plt.ylabel('mensualidades')
    axe.set_xticks(range(5 * 12), minor=True)
    plt.grid()
    plt.xlim(0, 12*5)
    plt.grid(which='minor', alpha=0.2)
    plt.subplots_adjust(hspace=0.07)
    plt.show()

interactive_plot = interactive(f, 
                               capital=(30_000, 40_000),  # prestamo de Vicen
                               obra=(30_000, 40_000),     # precio de la obra
                               pagos_extras=(500, 10000), # pagos al ayuntamiento, arquiteco etc...
                               aire_etc=(3000, 10000),    # aire acondicionado, camillas otro material
                               gastos=(200, 400),         # luz, agua, internet...
                               pacientes=(20, 40),        # numero de pacientes a la semana
                               consulta=(20, 40),         # precio (medio) de la consulta
                               sueldo=(1000, 2000),       # nomina de Alexis
                               mes_pago=(36, 48),         # mes en el que se acaba de pagar el bajo
                               devolucion=(100, 1000))    # devolucion del prestamo a Vicen

interactive_plot

for c in interactive_plot.children[:-1]:
    c.description_tooltip = tooltips[c.description]
    if c.description in ['obra', 'gastos', 'pagos_extras', 'aire_etc', 'devolucion Vicen', 'sueldo']:
        c.style.handle_color = 'firebrick'
    elif c.description in ['capital', 'consulta', 'pacientes']:
        c.style.handle_color = 'darkgreen'
#     else:
#         c.style.handle_color = 'red'

controls = HBox(interactive_plot.children[:-1], layout = Layout(flex_flow='row wrap'))

output = interactive_plot.children[-1]

output.layout.height = '750px'
display(VBox([controls, output]))
interactive_plot.update()